In [1]:
import pandas as pd 
df=pd.read_csv("Food Ingredients and Recipe Dataset with Image Name Mapping.csv")
df

,Unnamed: 0,Title,Ingredients,Instructions,Image_Name,Cleaned_Ingredients
0,0,Miso-Butter Roast Chicken With Acorn Squash Pa...,"['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher...","Pat chicken dry with paper towels, season all ...",miso-butter-roast-chicken-acorn-squash-panzanella,"['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher..."
1,1,Crispy Salt and Pepper Potatoes,"['2 large egg whites', '1 pound new potatoes (...",Preheat oven to 400°F and line a rimmed baking...,crispy-salt-and-pepper-potatoes-dan-kluger,"['2 large egg whites', '1 pound new potatoes (..."
2,2,Thanksgiving Mac and Cheese,"['1 cup evaporated milk', '1 cup whole milk', ...",Place a rack in middle of oven; preheat to 400...,thanksgiving-mac-and-cheese-erick-williams,"['1 cup evaporated milk', '1 cup whole milk', ..."
3,3,Italian Sausage and Bread Stuffing,"['1 (¾- to 1-pound) round Italian loaf, cut in...",Preheat oven to 350°F with rack in middle. Gen...,italian-sausage-and-bread-stuffing-240559,"['1 (¾- to 1-pound) round Italian loaf, cut in..."
4,4,Newton's Law,"['1 teaspoon dark brown sugar', '1 teaspoon ho...",Stir together brown sugar and hot water in a c...,newtons-law-apple-bourbon-cocktail,"['1 teaspoon dark brown sugar', '1 teaspoon ho..."
...,...,...,...,...,...,...
13496,13496,Brownie Pudding Cake,"['1 cup all-purpose flour', '2/3 cup unsweeten...",Preheat the oven to 350°F. Into a bowl sift to...,brownie-pudding-cake-14408,"['1 cup all-purpose flour', '2/3 cup unsweeten..."
13497,13497,Israeli Couscous with Roasted Butternut Squash...,"['1 preserved lemon', '1 1/2 pound butternut s...",Preheat oven to 475°F.\nHalve lemons and scoop...,israeli-couscous-with-roasted-butternut-squash...,"['1 preserved lemon', '1 1/2 pound butternut s..."
13498,13498,Rice with Soy-Glazed Bonito Flakes and Sesame ...,['Leftover katsuo bushi (dried bonito flakes) ...,"If using katsuo bushi flakes from package, moi...",rice-with-soy-glazed-bonito-flakes-and-sesame-...,['Leftover katsuo bushi (dried bonito flakes) ...
13499,13499,Spanakopita,['1 stick (1/2 cup) plus 1 tablespoon unsalted...,Melt 1 tablespoon butter in a 12-inch heavy sk...,spanakopita-107344,['1 stick (1/2 cup) plus 1 tablespoon unsalted...


In [2]:
# Example usage:
recipe_title = "Flash-Fried Finger-Lickin' Chicken"  # Title of the recipe
recipe_row = df[df['Title'] == recipe_title]
if not recipe_row.empty:
    recipe_ingredients_str = recipe_row['Ingredients'].iloc[0]  # Get ingredients as a string
    # Remove unwanted characters from the string representation of the list
    recipe_ingredients_str_cleaned = recipe_ingredients_str.replace('[', '').replace(']', '').replace("'", "")
    # Split the string into a list of ingredients
    recipe_ingredients_list = recipe_ingredients_str_cleaned.split(', ')
    # Join the list of ingredients into a single string with commas between each ingredient
    ingredients_str = ', '.join(recipe_ingredients_list)
    print(f"Ingredients of '{recipe_title}': {ingredients_str}")
else:
    print("Recipe not found.")


Ingredients of 'Flash-Fried Finger-Lickin' Chicken': 3 cups low-fat, low sodium chicken broth, Salt, 4 skinless, bone-in chicken thighs, 2 quarts grapeseed oil, 1 1/2 cups whole wheat flour, 1 tablespoon sweet paprika, 1 1/2 teaspoon celery salt, 1 tablespoon freshly ground black pepper, 2 cups low-fat buttermilk


In [6]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances

# Preprocess the ingredients column
def preprocess(text):
    # Remove punctuation and convert to lowercase
    return text.lower().replace('[^\w\s]', '')

# Apply preprocessing to ingredients
df['Ingredients'] = df['Ingredients'].apply(preprocess)

# Vectorize ingredients using TF-IDF
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df['Ingredients'])
# Perform LSI
lsa_model = TruncatedSVD(n_components=100, random_state=42)
lsa_matrix = lsa_model.fit_transform(tfidf_matrix)

def get_top_matching_recipes(input_ingredients, df_sel, top_n=5):
    # Preprocess input ingredients
    input_text = ' '.join(input_ingredients)
    input_tfidf = vectorizer.transform([input_text])
    input_lsa = lsa_model.transform(input_tfidf)

    # Calculate cosine similarity between input and recipes
    similarity_scores = cosine_similarity(input_lsa, lsa_matrix)
    
    #Calculate Euclidean distance between input and recipes 
    euclidean_scores = euclidean_distances(input_lsa,lsa_matrix)

    #combining the scores of both cosine and euclidean distance 
    combine_score = 0.5 * similarity_scores + 0.5 * (1 / (1 + euclidean_scores))
    
    # Get indices of top matching recipes
    top_indices = combine_score.argsort(axis=1).flatten()[-top_n:][::-1]

    # Get top matching recipes
    top_recipes = [(df_sel.iloc[i]['Title'], combine_score[0, i]) for i in top_indices]

    return top_recipes 


# Example usage:
input_ingredients = input("Enter ingredients separated by commas: ").split(',')
top_recipes = get_top_matching_recipes(input_ingredients, df)
print("\nTop 5 matching recipes based on input ingredients:")
for idx, recipe in enumerate(top_recipes,start=1):
    print(f"{idx}. {recipe[0]} (Similarity Score: {recipe[1]:.2f})")
    


<>:10: SyntaxWarning: invalid escape sequence '\w'
<>:10: SyntaxWarning: invalid escape sequence '\w'
/var/folders/qx/md9zh1j51yl5lbrztcqysd280000gn/T/ipykernel_4938/691887891.py:10: SyntaxWarning: invalid escape sequence '\w'
  return text.lower().replace('[^\w\s]', '')



Top 5 matching recipes based on input ingredients:
1. Heat oven to 375°F. Heat oil in a large ovenproof sauté pan over medium-high heat. Season chicken with salt, pepper and herbes de Provence. Dust with flour; tap off excess. Cook, meaty side down, 5 minutes. Turn over; cook 1 minute. Add garlic and shallot; stir 30 seconds. Add wine; cook 1 minute. Add stock; cook 30 seconds. Add lemon juice. Transfer pan to oven. Bake until chicken is cooked through, 9 to 13 minutes. Transfer chicken to cutting board. Place pan over medium heat; simmer sauce 1 minute. Add butter; swirl pan until butter incorporates and sauce thickens slightly. Cut each breast into 4 slices; transfer each to a plate; top with sauce and garlic. Serve with spinach ; garnish with zest, if desired. (Similarity Score: 0.61)
2. Heat 2 tablespoons olive oil in heavy large skillet over medium-high heat. Add 2 tablespoons chopped shallots and 2 cups rhubarb; sauté until soft, about 5 minutes. Stir in lemon juice and 1 teaspo